In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=1)

import missingno as msno    # 누락값 표시
import warnings
import mglearn
warnings.filterwarnings('ignore')  # 워닝 무시
from sklearn.model_selection import train_test_split
%matplotlib inline

## 교차 검증(cross-validation)

#### k-fold cross-validation

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris = load_iris()
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target, cv=3)
print(scores)
print(scores.mean())

[0.98 0.96 0.98]
0.9733333333333333


In [58]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
stkfold = StratifiedKFold(n_splits=5)
kfold = KFold(n_splits=5, shuffle=True,)
print(cross_val_score(logreg, iris.data, iris.target, cv=stkfold).mean())
cross_val_score(logreg, iris.data, iris.target, cv=kfold).mean()

0.9733333333333334


0.9666666666666668

#### LOOCV(LeaveOneOut)

In [63]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print(len(scores), '{:2f}'.format(scores.mean()))

150 0.966667


#### 임의 분할 교차 검증(ShuffleSplit)

In [81]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(train_size=.5, test_size=.3, n_splits=10, random_state=0)
score = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
score

array([0.97777778, 0.93333333, 0.95555556, 0.88888889, 0.97777778,
       0.93333333, 0.97777778, 0.97777778, 1.        , 0.97777778])

#### 그룹별 교차 검증(GroupKFold)

In [80]:
from sklearn.model_selection import GroupKFold
from sklearn.datasets import make_blobs
# 인위적 데이터셋 생성
X, y = make_blobs(n_samples=13, random_state=0)
groups = [0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4]
scores = cross_val_score(logreg, X, y, groups, cv=GroupKFold(n_splits=3))
scores

array([0.75, 0.6 , 1.  ])